In [1]:
pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import csv
import numpy as np
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import fuzz
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk import word_tokenize
import pickle


df=pd.read_csv(r'YourDataset.csv')
df

In [ ]:
X=['TrainingSession_title','course_type']#the key points on which the training is recommended
for X in X:
    df[X]=df[X].fillna(' ')#if any data containg Null or no value it would be rplaced by blank space
def all(row):
    try:
        return row['TrainingSession_title']+" "+row["course_type"] 
    except: print("error", row)#giving out the error fields.
df["all"]= df.apply(all,axis=1)
df.head()

In [ ]:
tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1,2), min_df = 0, stop_words = 'english')
tfidf_matrix = tf.fit_transform(df['TrainingSession_title'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim


In [ ]:
# create a function to find the closest title
def matching_score(a,b):
   return fuzz.ratio(a,b)

In [ ]:
def get_TrainingSession_title_from_index(index):
   return df[df.index == index]['TrainingSession_title'].values[0],df[df.index == index]['course_link'].values[0]

def get_index_from_TrainingSession_title(TrainingSession_title):
   return df[df.TrainingSession_title == TrainingSession_title].index.values[0]

# a function to return the most similar title to the words a user type
def find_closest_title(TrainingSession_title):
   leven_scores = list(enumerate(df['TrainingSession_title'].apply(matching_score, b=TrainingSession_title)))
   sorted_leven_scores = sorted(leven_scores, key=lambda x: x[1], reverse=True)
   closest_title,a = get_TrainingSession_title_from_index(sorted_leven_scores[0][0])
   distance_score = sorted_leven_scores[0][1]
   return closest_title, distance_score

In [ ]:
fav_list = []
#the list for user name 
username = []
searchlist = []
def recommender(Training_user_likes, how_many):
     closest_title, distance_score = find_closest_title(Training_user_likes)
     recommender.ct = closest_title
     recommender.ds = distance_score
    # When a user does not make misspellings
     if distance_score == 100:
        Training_index,a = get_TrainingSession_title_from_index (closest_title)
        Training_list = list(enumerate(cosine_sim[int(Training_index)]))
        print(closest_title)

       # remove the typed course itself
        similar_Training = list(filter(lambda x:x[0] != int(Training_index), sorted(Training_list,key=lambda x:x[1], reverse=True))) 

        print(' Similar courses '+'\033[1m'+str(closest_title)+'\033[0m'+'.\n')

        for i,s in similar_Training[:how_many]:
         print(get_TrainingSession_title_from_index(i)) 
         print('Simialr courses '+'\033[1m'+str(closest_title)+'\033[0m'+'.\n')

     else:
      Training_index = get_index_from_TrainingSession_title(closest_title)
      Training_list = list(enumerate(cosine_sim[int(Training_index)]))
      similar_Training= list(filter(lambda x:x[0] != int(Training_index), sorted(Training_list,key=lambda x:x[1], reverse=True)))
      print(' \n')
      print(closest_title)
      
      for i,s in similar_Training[:how_many]:
         print(get_TrainingSession_title_from_index(i))
         fav_list.append(get_TrainingSession_title_from_index(i))



In [ ]:
def user_in():
    
    name = input(" Enter your name: \n")
    
    if any(name in subl for subl in username) == True:

        print("\n Sorry, the name is already Exist")     
        
        
    else :
      recomentation = 5
      search = input(" \n What are your favorite course topics? \n")
      user_in.sea = search

      print(name, end =", ")
      recommender(search,recomentation)
      user_info(name,search)

In [ ]:
def login():
    name = input("Enter your name  \n")
    recomentation = 8
    if name in username:
        nindex =username.index(name)
        print(" Welcome back")
        print(name, end =", ")
        old_search = searchlist[nindex]
        print("Based in your previous search, you may want to watch these courses", old_search)
        recommender(old_search,recomentation)
        user_info(name,old_search)        
    else :
        print("Sorry, incorrect name! ")

In [ ]:
def user_info(name,fav):
    #append each element 
    username.append(name)
    searchlist.append(fav)
    k=0
    #each user will be appended 
    file1 = open("myfile.txt","a",encoding = 'utf-8')
    file1.write("\n Username:\t "+name+"\n ")
    file1.write(" preferences: \t"+ fav+"\n  " )
    fav_list.reverse()
    for i,j in fav_list:
        file1.write(str(i)+" "+str(j)+ "\n")
        k=k+1
        if(k ==5):
            break;

In [ ]:
user_in()

In [ ]:
login()


In [ ]:
print ('The accuracy between user input and the recommendation is :', recommender.ds/100)
pred = []
in_put = []

in_put.append(user_in.sea)
print('Input User is:', in_put) 

pred.append(recommender.ct)
print('The Result Of Recommendation System is:',pred) 

from difflib import SequenceMatcher
ratio = SequenceMatcher(None, recommender.ct, user_in.sea).ratio()
print('Ratio of Similarity is:', ratio)